# CS/ECE/ISyE 524 - Spring 2017 - Homework 8 - Solutions
### Prepared by: Laurent Lessard

## 1. Voting

Governor Blue of the state of Berry is attempting to get the state
legislator to gerrymander Berry's congressional districts.  The state
consists of ten cities, and the numbers of registered Republicans and
Democrats (in thousands) in each city are shown below

| City | Republicans | Democrats |
|:----:|:-----------:|:---------:|
|1     |  80         |    34     |
|2     |  60         |    44     |
|3     |  40         |    44     |
|4     |  20         |    24     |
|5     |  40         |    114    |
|6     |  40         |    64     |
|7     |  70         |    14     |
|8     |  50         |    44     |
|9     |  70         |    54     |
|10    |  70         |    64     |

Berry has five congressional representatives.  To form the five congressional
districts, cities must be grouped together according to the following
restrictions:

- Districts cannot subdivide cities; all voters in a city must be in the same district.
- Each district must contain between 150,000 and 250,000 voters (there are no independent voters).  

Governor Blue is a Democrat. Assume 100\% voter turnout and that each voter always votes according to their registered party.  Formulate and solve an optimization problem to help Governor Blue maximize the number of congressional districts that have a Democratic majority.

In [5]:
using JuMP, NamedArrays

parties = [:rep, :dem]
districts = [:A, :B, :C, :D, :E];  # five districts
cities = collect(1:10)

# first column is republican, second column is democrat
# columns are cities
data = [ 80 34
60 44
40 44
20 24
40 114
40 64
70 14
50 44
70 54
70 64 ];

counts = NamedArray( data, (cities,parties))


m = Model()

# does district i contain city j?
@variable(m, x[districts,cities], Bin)

# do the democrats win this district?
@variable(m, z[districts], Bin)

# each district must contain between 150k and 250k total voters
for i in districts
    @constraint(m, 150 <= sum( (counts[j,:rep] + counts[j,:dem])*x[i,j] for j in cities ) <= 250 )
end

# each city belongs to exactly one district
for j in cities
    @constraint(m, sum( x[i,j] for i in districts) == 1)
end

# since there are at most 250k per district, there are at most 250k republicans.
# since there are at least 150k per district, there are at least 150 democrats.
# therefore, #reps - #dems <= 250 - 150 = 100.
# if z=1, there must be more democrats than republicans in the district.
for i in districts
    @constraint(m, sum( (counts[j,:rep] - counts[j,:dem])*x[i,j] for j in cities) <= 100*(1-z[i]) )
end

@objective(m, Max, sum(z))

solve(m)

sol = NamedArray( zeros(5,10), (districts,cities))
for i in districts
    for j in cities
        sol[i,j] = getvalue(x[i,j])
    end
end

println("The assignment of cities to districts is:")
show(sol)
println()

println("The final tally of republicans and democrats in each district is:")
show(sol*counts)

println()
println("The democrats win a total of ", Int(getobjectivevalue(m)), " districts (out of 5)")

The assignment of cities to districts is:
5×10 Named Array{Float64,2}
A ╲ B │   1    2    3    4    5    6    7    8    9   10
──────┼─────────────────────────────────────────────────
A     │ 0.0  0.0  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0
B     │ 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
C     │ 0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
D     │ 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
E     │ 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0
The final tally of republicans and democrats in each district is:
5×2 Named Array{Float64,2}
A ╲ B │   rep    dem
──────┼─────────────
A     │ 110.0  112.0
B     │ 150.0   88.0
C     │ 100.0  108.0
D     │  40.0  114.0
E     │ 140.0   78.0
The democrats win a total of 3 districts (out of 5)


# 2. Paint production

As part of its weekly production, a paint company produces five batches of paints, always the same, for some big clients who have a stable demand.  Every paint batch is produced in a single production process, all in the same blender that needs to be cleaned between each batch.  The durations of blending paint batches 1 to 5 are 40, 35, 45, 32 and 50 minutes respectively.  The cleaning times depend of the colors and the paint types.  For example, a long cleaning period is required if an oil-based paint is produced after a water-based paint, or to produce white paint after a dark color.  The times are given in minutes in the following matrix $A$ where $A_{ij}$ denotes the cleaning time after batch $i$ if it is followed by batch $j$.

$$
A = \begin{bmatrix}
 0&11& 7&13&11 \\
 5& 0&13&15&15 \\
13&15& 0&23&11 \\
 9&13& 5& 0& 3 \\
 3& 7& 7& 7& 0
\end{bmatrix}
$$

Since the company has other activities, it wishes to deal with this weekly production in the shortest possible time (blending and cleaning).  What is the corresponding order of paint batches?  The order will be applied every week, so the cleaning time between the last batch of one week and the first of the following week needs to be accounted for in the total duration of cleaning.

In [12]:
# This is a traveling salesman problem! cities correspond to paint batches, and the time it takes to travel
# from one city to the next is the time it takes to clean the blender in between two particular batches.

# Because the cleaning time between the last batch of one week and the first batch of the following week needs
# to be accounted for, this is EXACTLY a traveling salesman problem!

# A[i,j] is the time it takes to travel j --> i.
A = [  0  11   7  13  11
       5   0  13  15  15
      13  15   0  23  11
       9  13   5   0   3
       3   7   7   7   0 ]

# time it takes to blend a particular batch. This information isn't important, because we need to blend each batch
# every week no matter what order they're blended in. So this is a fixed cost!
b = [ 40, 35, 45, 32, 50 ]

using JuMP
m = Model()
n = length(b)  # number of paints

# x[i,j] = 1 if we blend i immediately after having blended j.
@variable(m, x[1:n,1:n], Bin)

# can't flow from a node to itself
@constraint(m, nzdiag[i=1:n], x[i,i] == 0 )

# flow constraints (exactly one flow out of each )
@constraint(m,  inflows[i=1:n], sum( x[i,j] for j=1:n ) == 1 )
@constraint(m, outflows[j=1:n], sum( x[i,j] for i=1:n ) == 1 )

# additional variables and constraints for MTZ formulation
@variable(m, u[1:n] >= 0)
@constraint(m, MTZ[i=1:n, j=2:n], u[i] - u[j] + n*x[i,j] <= n-1)

@objective(m, Min, sum( x[i,j]*A[i,j] for i=1:n, j=1:n ))

solve(m)

println("Optimal ordering is:")
            X = getvalue(x);
            for i = 1:n
                println(find(X[i,:]))
            end
#            println(getvalue(x))
println("and the minimum cleanup time is: ", getobjectivevalue(m), " min, (plus ", sum(b), " min of blending)")

Optimal ordering is:
[4]
[1]
[5]
[3]
[2]
and the minimum cleanup time is: 41.0 min, (plus 202 min of blending)


# 3. The Queens problem
You are given a standard 8 x 8 chess board. The following problems involve placing queens on the board such that certain constraints are satisfied. For each of the following problems, model the optimization task as an integer program, solve it, and show what an optimal placement of queens on the board looks like.

**(a)** Find a way to place 8 queens on the board so that no two queens threaten each other. We say that two queens \emph{threaten} each other if they occupy the same row, column, or diagonal. Show what this placement looks like.
	
**(b)** Repeat part (a) but this time find a placement of the 8 queens that has point symmetry. In other words, find a placement that looks the same if you rotate the board 180&deg;.
	
**(c)** What is the smallest number of queens that we can place on the board so that each empty cell is threatened by at least one queen? Show a possible optimal placement.
	
**(d)** Repeat part (c) but this time find a placement of the queens that also has point symmetry. Does the minimum number of queens required change? Show a possible optimal placement.

In [1]:
# helper function to print a chess board
function printBoard(X,n)
    print("+")
    for i in 1:n
        print("---+")
    end
    print("\n")
    
    for i in 1:n
        print("| ")
        for j in 1:n
            X[i,j] == 1 ? print("X") : print(" ")
            print(" | ")
        end
        print("\n")
        print("+")
        for j in 1:n
            print("---+")
        end
        print("\n")
    end
end

printBoard (generic function with 1 method)

In [7]:
# solve the 8 queens problem (queens don't threaten each other)

using JuMP

n = 6

m = Model()

@variable(m, x[1:n,1:n], Bin)

for i in 1:n
    @constraint(m, sum(x[i,j] for j in 1:n) == 1)  # exactly one per row
    @constraint(m, sum(x[j,i] for j in 1:n) == 1)  # exactly one per column
end
for i in 1:n
    @constraint(m, sum(x[i+k,1+k] for k in 0:n-i) <= 1)  # "\" upper diagonal
    @constraint(m, sum(x[i-k,1+k] for k in 0:i-1) <= 1)  # "/" upper diagonal
end
for j in 2:n
    @constraint(m, sum(x[1+k,j+k] for k in 0:n-j) <= 1)  # "\" lower diagonal
    @constraint(m, sum(x[n-k,j+k] for k in 0:n-j) <= 1)  # "/" lower diagonal
end

# SYMMETRY CONSTRAINT --- REMOVE THIS FOR NON-SYMMETRIC SOLUTIONS
for i in 1:n
    for j in i:n
        @constraint(m, x[i,j] == x[n-i+1,n-j+1] )
    end
end

status = solve(m)
X = getvalue(x)
println("A solution is: ")
printBoard(X,n)

A solution is: 
+---+---+---+---+---+---+
|   |   |   |   | X |   | 
+---+---+---+---+---+---+
|   |   | X |   |   |   | 
+---+---+---+---+---+---+
| X |   |   |   |   |   | 
+---+---+---+---+---+---+
|   |   |   |   |   | X | 
+---+---+---+---+---+---+
|   |   |   | X |   |   | 
+---+---+---+---+---+---+
|   | X |   |   |   |   | 
+---+---+---+---+---+---+


In [3]:
# solve the min queen problem (all squares are threatened)

using JuMP

n = 8

m = Model()

@variable(m, x[1:n,1:n], Bin)

for i = 1:n
    for j = 1:n
        @constraint(m, sum(x[i,:]) + sum(x[:,j])
            + sum(x[i+k,j+k] for k = max(1-i,1-j):min(n-i,n-j))
            + sum(x[i+k,j-k] for k = max(1-i,n-j):min(n-i,1-j)) >= 1)
    end
end

@objective(m, Min, sum(x))

status = solve(m)
X = getvalue(x)
println("A solution is: ")
printBoard(X,n)

A solution is: 
+---+---+---+---+---+---+---+---+
| X |   |   |   |   |   |   |   | 
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   | X |   | 
+---+---+---+---+---+---+---+---+
|   |   |   |   | X |   |   |   | 
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   | X | 
+---+---+---+---+---+---+---+---+
|   |   |   |   |   | X |   |   | 
+---+---+---+---+---+---+---+---+


In [2]:
# solve the min queen problem (with symmetry constraint)

using JuMP

n = 8

m = Model()

@variable(m, x[1:n,1:n], Bin)

for i = 1:n
    for j = 1:n
        @constraint(m, sum(x[i,:]) + sum(x[:,j])
            + sum(x[i+k,j+k] for k = max(1-i,1-j):min(n-i,n-j))
            + sum(x[i+k,j-k] for k = max(1-i,n-j):min(n-i,1-j)) >= 1)
    end
end

# SYMMETRY CONSTRAINT --- REMOVE THIS FOR NON-SYMMETRIC SOLUTIONS
for i in 1:n
    for j in i:n
        @constraint(m, x[i,j] == x[n-i+1,n-j+1] )
    end
end

@objective(m, Min, sum(x))

status = solve(m)
X = getvalue(x)
println("A solution is: ")
printBoard(X,n)

A solution is: 
+---+---+---+---+---+---+---+---+
|   | X | X |   |   |   |   |   | 
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 
+---+---+---+---+---+---+---+---+
| X |   |   |   |   |   |   |   | 
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   | X | 
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 
+---+---+---+---+---+---+---+---+
|   |   |   |   |   | X | X |   | 
+---+---+---+---+---+---+---+---+
